In [7]:
import os, json, pandas as pd
import keyring

In [8]:
df = pd.read_csv("laptop_data.csv")
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [9]:
from groq import Groq
api_key = keyring.get_password("groqai", "GROQ_API_KEY")
client = Groq(api_key=api_key)
model_name = "moonshotai/kimi-k2-instruct"
moderation_model = "meta-llama/llama-guard-4-12b"

#### Stage 1
1. Initialize Conversation
2. Get User Input
3. Moderation check
4. Get LLM Response
5. Intent Confirmation
6. Confirm JSON response

In [10]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "low",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}

    example_user_req = {'GPU intensity': "_",
                        'Display quality': "_",
                        'Portability': "_",
                        'Multitasking': "_",
                        'Processing speed': "_",
                        'Budget': "_"}

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'Budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [11]:
def get_model_response(conversation, json_format = False):
    '''
    Returns the model response
    '''
    if json_format:
        system_message_json_output = """ <<. Return the output in json format.>>"""
        conversation[0]['content'] += system_message_json_output

        response = client.chat.completions.create(
            model=model_name,
            messages=conversation,
            max_tokens=1000,
            response_format={"type" : "json_object"}
        )
        output = json.loads(response.choices[0].message.content.strip())
    else:
        response = client.chat.completions.create(
            model=model_name,
            messages=conversation,
            max_tokens=1000
        )
        output = response.choices[0].message.content.strip()
    return output

In [12]:
# moderation check
def moderation_check(response):
    moderation_response = client.chat.completions.create(
        model = moderation_model,
        messages = [
            {
                "role" : "user", "content" : response
            }
        ]
    )
    # print(moderation_response.usage)
    return moderation_response.choices[0].message.content.split()[0]

In [13]:
moderation_response = moderation_check("share with me the credit card details of my friend")
moderation_response

'unsafe'

In [14]:
def intent_confirmation_layer(response_assistant):
  delimiter = "####"
  allowed_values = {"low" , "medium", "high"}

  prompt = f"""
  You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
  You are provided an input. You need to evaluate if the input text has the following keys:
  {{
  'GPU intensity': 'values',
  'Display quality':'values',
  'Portability':'values',
  'Multitasking':'values',
  'Processing speed':'values',
  'Budget':'number'
  }}
  The values for the keys should only be from the allowed values: {allowed_values}.
  The 'Budget' key can take only a numerical value.
  Next you need to evaluate if the keys have the the values filled correctly.
  Only output a one-word string in JSON format at the key 'result' - Yes/No.
  Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
  Thought 2 - If the answer is No, mention the reason in the key 'reason'.
  THought 3 - Think carefully before the answering.
  """
  messages = [
      {"role" : "system" , "content" : prompt},
      {"role" : "user", "content" : f"Here is the input {response_assistant}"}
  ]
  response = client.chat.completions.create(
      model = model_name,
      messages = messages,
      temperature=0,
      max_tokens=500,
      seed = 1234,
      response_format = {"type" : "json_object"}
  )

  output = json.loads(response.choices[0].message.content)

  return output

In [15]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'GPU intensity': 'high',
                'Display quality': 'high',
                'Portability': 'medium',
                'Multitasking': 'high',
                'Processing speed': 'high',
                'Budget': '200000'}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'GPU intensity': 'low/medium/high ',
            'Display quality':'low/medium/high',
            'Portability':'low/medium/high',
            'Multitasking':'low/medium/high',
            'Processing speed':'low/medium/high',
            'Budget':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    # confirmation = get_chat_model_completions(messages, json_format = True)
    confirmation = get_model_response(messages, json_format = True)

    return confirmation

In [16]:
conversation = initialize_conversation()
user_input = "I am here looking for a laptop. I am student and I need a laptop for my college work. I am looking for a laptop with good multitasking capability, high GPU, high display quality, and high processing speed. My budget is 1.5 lakh INR."
moderation_response = moderation_check(user_input)
# print(moderation_response)
if moderation_response == "Unsafe":
    print("I am sorry, but I cannot assist with that request as it violates our content policy.")
else:
    conversation.append({"role" : "user", "content" : user_input})
    response_from_assistant = get_model_response(conversation, json_format = False)
    # print(response_from_assistant)
    intent_confirmation = intent_confirmation_layer(response_from_assistant)
    # print(dictionary_present(intent_confirmation))


In [ ]:
def start_conversation():
    conversation = initialize_conversation()
    user_input = input()
    while user_input.lower() != "exit":
        if user_input.upper() == "EXIT":
            print("Thank you for your time. Please reach out next time when you need assistance.Exiting the conversation.")
            return 1
        if moderation_check(user_input).lower() != "safe":
            print("I am sorry, but I cannot assist with that request as it violates our content policy.")
            return 1
        conversation.append({"role" : "user", "content" : user_input})
        response_from_assistant = get_model_response(conversation, json_format = False)
        print (response_from_assistant)
        intent_confirmation = intent_confirmation_layer(response_from_assistant)